Our task is to obtain hockey league results from publicly available sources 
and then determine initial betting odds, which will be updated as new data comes in.

# Introduction

The first step in our task is to obtain the data necessary for analysis. Since our company is in the early stages of development and does not have its own database, we intend to use publicly available resources.  
  
For this purpose, we have been recommended the website [Scrape This Site](https://www.scrapethissite.com/pages/forms/). However, before we start downloading data, it is important to carefully review the [FAQ](https://www.scrapethissite.com/faq/) section on the site. Particular attention should be paid to the restrictions on the number of requests, which is crucial for our solution.  
  
It is expected that after executing the code contained in this notebook, the `data/raw/` folder will be populated with data, which will serve as the source for the next stage of the project.

Clients may only make a maximum of one request per second
Clients must send an identifying user agent
Clients must respect this site's robots.txt file

# Notebook Configuration

## Importing Required Libraries

In [1]:
import selenium
print(selenium.__version__)


4.37.0


# THIS IS HOW WE GET READY OUR TOOLS FOR WEBSRAPING

In [2]:
import requests # this will download the website
from bs4 import BeautifulSoup # this will make it possible to find particular elements on the website

## Driver and Selenium Configuration

# DO WE NEED SELENIUM FOR SCRAPING THIS SITE? 
No! We dont need selenium for scraping this site: https://www.scrapethissite.com/pages/forms/
Why? The website is static and is not using javascript. Its contents are embedded directly in html code, therefore, we can scrape it only by using requests. 

# Fetching Website Content

This section of the notebook contains code for fetching website content. To properly execute the task, consider the following steps:  
- Ensure all available data on the site has been fetched by checking if there are additional data pages.  
- Locate the data of interest on the page using `html` inspection tools.  
- Navigate between subsequent data pages using browser mechanisms or by analyzing the `url` structure.  
  
> Remember to respect the query limits specified in the `FAQ`!  
  
Save the fetched data to the folder `data/raw/hockey_teams_page_{page_number}.html`. At this stage, we are retrieving data without processing it - analysis will be performed later.  
  
To fetch the `html` content of the page, you can use `browser.page_source`. Make sure the browser tool configuration (e.g., Selenium) is ready for use.  
  
> (Optional) If there are multiple pages to fetch, use the [zfill](https://www.programiz.com/python-programming/methods/string/zfill) function to maintain order in file names by adding leading zeros to the page numbers.



In [7]:

r = requests.get("http://www.scrapethissite.com/pages/forms/?per_page=1000") ##fetching html contents
# note this part of code: "per_page=1000" - thus, we have all contents from the website scraped on one page
soup = BeautifulSoup(r.text, "html.parser") #using beautiful soup to be able to get individual elements from the site

soup
print (soup.prettify()) # returns a nice (prettified) code 
#prettify je textová funkce a funguje tím pádem jen v kombinaci s printem
# now we have all 600 teams in html code

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping
  </title>
  <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
  <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
  <meta con

# Locate the data of interest on the page using html inspection tools.
? Co to znamená: 
- Najít HTML strukturu, kde se data nachází
- Zjistit, jaké tagy a třídy se používají (div, span, class="...")
- Vytvořit správný CSS selektor pro BeautifulSoup (.team-name, div.team, apod.)



In [11]:
#first, we select "teams" in html structure: 

teams = soup.select("tr.team")

# then, we iterate over each team to get statistics using html selectors:

for team in teams: 
    name = team.select_one("td.name").text.strip()
    year = team.select_one("td.year").text.strip()
    wins = team.select_one("td.wins").text.strip()
    losses = team.select_one("td.losses").text.strip()
    ot_losses = team.select_one("td.ot-losses").text.strip()
    win_pct = team.select_one("td.pct").text.strip()
    gf = team.select_one("td.gf").text.strip()
    ga = team.select_one("td.ga").text.strip()
    diff = team.select_one("td.diff").text.strip()

    print(f"{name} ({year}): {wins}W, {losses}L, {ot_losses}OTL, Pct: {win_pct}, GF: {gf}, GA: {ga}, Diff: {diff}")


Boston Bruins (1990): 44W, 24L, OTL, Pct: 0.55, GF: 299, GA: 264, Diff: 35
Buffalo Sabres (1990): 31W, 30L, OTL, Pct: 0.388, GF: 292, GA: 278, Diff: 14
Calgary Flames (1990): 46W, 26L, OTL, Pct: 0.575, GF: 344, GA: 263, Diff: 81
Chicago Blackhawks (1990): 49W, 23L, OTL, Pct: 0.613, GF: 284, GA: 211, Diff: 73
Detroit Red Wings (1990): 34W, 38L, OTL, Pct: 0.425, GF: 273, GA: 298, Diff: -25
Edmonton Oilers (1990): 37W, 37L, OTL, Pct: 0.463, GF: 272, GA: 272, Diff: 0
Hartford Whalers (1990): 31W, 38L, OTL, Pct: 0.388, GF: 238, GA: 276, Diff: -38
Los Angeles Kings (1990): 46W, 24L, OTL, Pct: 0.575, GF: 340, GA: 254, Diff: 86
Minnesota North Stars (1990): 27W, 39L, OTL, Pct: 0.338, GF: 256, GA: 266, Diff: -10
Montreal Canadiens (1990): 39W, 30L, OTL, Pct: 0.487, GF: 273, GA: 249, Diff: 24
New Jersey Devils (1990): 32W, 33L, OTL, Pct: 0.4, GF: 272, GA: 264, Diff: 8
New York Islanders (1990): 25W, 45L, OTL, Pct: 0.312, GF: 223, GA: 290, Diff: -67
New York Rangers (1990): 36W, 31L, OTL, Pct: 0.

In [15]:

import os

# Nastav cestu ke složce (vedle notebooks)
folder_path = "../data/raw"
os.makedirs(folder_path, exist_ok=True)

# Název souboru
file_name = f"hockey_teams.html"
full_path = f"{folder_path}/{file_name}"

# Stáhni HTML ze stránky
import requests
url = f"http://www.scrapethissite.com/pages/forms/?page_num={page_number}&per_page=1000"
r = requests.get(url)

# Ulož HTML do souboru
with open(full_path, "w", encoding="utf-8") as f:
    f.write(r.text)

print(f"Soubor uložen do: {full_path}")


Soubor uložen do: ../data/raw/hockey_teams.html


# Summary

Downloading raw data from our source has reduced the risk of problems stemming from site updates during the extraction process. This method also offers an additional benefit: it allows easy access to the data in its original form, which is crucial if reprocessing is needed.

In the next step, we will focus on extracting the necessary information from the `html` pages, which is essential for conducting the analysis.

Zpětná vazba
- selenium se potom už nikde nenačítá - klidně mohlo být vynecháno
- dvakrát se volá request.get, lepší je dát URL do jedné proměnné a použít to konzistentně
- v části kódu pro ukládání HTML je proměnná page_number, která potom není nikde použita

Jinak v pořádku splněno podle zadání. 